In [1]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np

main_url = "http://www.boattrader.com" #/listing/2006-sea-ray-320-sundancer-103069603/

In [2]:

def string_check(value):
    if value is None:
        return "" 
    return str(value.text)

def from_table(soup,name):
    
    data = list()
    #columns = ['Class','Category']#'Year','Make','Length','Propulsion','Type','Hull Material','Fuel Type','Location']
    if(soup.find('div',{'id':name}) is not None):
        table = soup.find('div',{'id':name}).find('table')
        rows = table.find_all('tr')
        for row in rows:
            if(name =='measurements'):
                index = string_check(row.find('dt'))
                val  = string_check(row.find('dd')) 
            else:
                index = string_check(row.find('th'))
                val  = string_check(row.find('td'))
            
            #if(index in columns):
            data.append((index,val))
            
    return data
        

def getsoup(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r,'html.parser')
    return soup

def check_element(element):
    return element if element is not None else " "


def scrape_page(s):
    
    a = s.find("h1",attrs={"class":"bd-name"})
    b = s.find("div",attrs={"class":"bd-price-location"})
    data = []
    df = pd.DataFrame()
    if a is not None:
        year =  str(check_element(a.find('span',{"class":"bd-year"})).get_text())
        make =  str(check_element(a.find('span',{"class":"bd-make"})).get_text())
        model = str(check_element(a.find('span',{"class":"bd-model"})).get_text())
        data.append(("year-primary",year))
        data.append(("make-primary",make))
        data.append(("model",model))
        
    if b is not None:                
        price = str(check_element(b.find('span',{"class":"bd-price contact-toggle"})).get_text()).split()[0]
        data.append(("price",price))
    #else:
    #    data.append(" ")
    content = s.find('div',{'class':'detail-tbl'})
    if content is not None:
        data  = data + from_table(content,'boat-details')
        #data  = data + from_table(content,'measurements')
        #data  = data + from_table(content,'propulsion')
        data  = data + from_table(content,'other-details')
        #data = pd.DataFrame.from_records(data)
        list1,list2 = zip(*data)
        df  = pd.DataFrame(list(list2),index = (list(list1))).transpose()
    print(df)
    return df

def scrape_full_page(url):
    soup = getsoup(url)
    a = soup.find_all("a", attrs={"data-reporting-click-listing-type":"standard listing"})
    links = [str(main_url+x['href']) for x in a]
    data = pd.DataFrame()

    for link in links:
        s = getsoup(link)
        data = pd.concat([data,scrape_page(s)], axis=0, ignore_index=True) 
    return data
    
    
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     from_table(details,'boat-details')


In [3]:
def run(zip,radius):
    url = "http://www.boattrader.com/search-results/NewOrUsed-any/Type-any/Category-all/Zip-"+zip+"/Radius-"+radius+"/Sort-Length:DESC"
    df = scrape_full_page(url)
    soup = getsoup(url)
    link = soup.find('a',{'class':'next'})
    while link.find('href') is not None:
        url = str(main_url+link['href'])
        df = pd.concat([df,scrape_full_page(url)], axis=0, ignore_index=True) 
        link = soup.find('a',{'class':'next'})
    df.to_csv(zip+"_"+radius+".csv")
#columns = ['Year','Make','Model','Price','Class','Category','Year1','Make1','Length','Propulsion','Type','Hull Material','Fuel Type','Location']
#t = pd.DataFrame(df,columns = columns)


In [4]:
run('10044','1')

  year-primary     make-primary         model       price  Class  \
0         2012  Carolina Custom  60 Sportfish  $1,950,000  Power   

             Category  Year             Make Length Propulsion Type  \
0   Convertible Boats  2012  Carolina Custom    60'    Twin Inboard   

  Hull Material Fuel Type    Location  
0     Composite    Diesel  Queens, NY  
  year-primary make-primary      model     price  Class   Category  Year  \
0         2015  Chris-Craft  Launch 36  $499,000  Power   Bowrider  2015   

          Make Length Propulsion Type Hull Material Fuel Type  \
0  Chris-Craft    36'           Other    Fiberglass    Diesel   

          Location Hull Shape  
0  Miami Beach, FL      Other  
  year-primary make-primary       model    price  Class  \
0         2005       Larson  Cabrio 310  $54,900  Power   

                 Category  Year    Make Length Propulsion Type Hull Material  \
0   Cruisers, Cuddy Cabin  2005  Larson    31'           Other    Fiberglass   

  Fuel Type 